In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE228783"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE228783"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE228783.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE228783.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE228783.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Inter-patient heterogeneity in the hepatic ischemia-reperfusion injury transcriptome: implications for research and diagnostics"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: liver'], 1: ['patient: 1', 'patient: 2', 'patient: 3', 'patient: 4', 'patient: 5', 'patient: 6', 'patient: 7', 'patient: 8', 'patient: 9', 'patient: 10', 'patient: 11', 'patient: pat 1', 'patient: pat 2', 'patient: pat 3', 'patient: pat 4', 'patient: pat 5', 'patient: pat 6', 'patient: pat 7', 'patient: pat 8', 'patient: pat 9', 'patient: pat 10', 'patient: pat 11', 'patient: pat 12', 'patient: pat 13', 'patient: pat 14', 'patient: pat 15', 'patient: pat 16', 'patient: pat 17', 'patient: pat 18', 'patient: pat 19'], 2: ['disease: CRC Met', 'disease: CCC', 'disease: HCC', 'disease: other'], 3: ['steatosis: NA', 'steatosis: <5%', 'steatosis: >20%'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the metadata, this dataset seems to be about liver ischemia-reperfusion injury
# and likely contains gene expression data for different time points and disease states
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Liver Cancer), we can use the disease information in row 2
trait_row = 2

# Age is not available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert disease status to binary (1 for liver cancer, 0 for other)
    """
    if not value or isinstance(value, float) and np.isnan(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # HCC stands for Hepatocellular Carcinoma, which is primary liver cancer
    if value == "HCC":
        return 1
    # CRC Met (colorectal cancer metastasis), CCC (cholangiocarcinoma), 
    # and other conditions are not primary liver cancer
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous numeric format"""
    if value is None or isinstance(value, float) and np.isnan(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None or isinstance(value, float) and np.isnan(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value in ["male", "m"]:
        return 1
    elif value in ["female", "f"]:
        return 0
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Clinical data seems to be available
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Assuming clinical_data is available from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)


Preview of clinical data:
{'GSM7136321': [0.0], 'GSM7136322': [0.0], 'GSM7136323': [0.0], 'GSM7136324': [0.0], 'GSM7136325': [0.0], 'GSM7136326': [0.0], 'GSM7136327': [0.0], 'GSM7136328': [0.0], 'GSM7136329': [0.0], 'GSM7136330': [0.0], 'GSM7136331': [0.0], 'GSM7136332': [0.0], 'GSM7136333': [0.0], 'GSM7136334': [0.0], 'GSM7136335': [0.0], 'GSM7136336': [0.0], 'GSM7136337': [0.0], 'GSM7136338': [0.0], 'GSM7136339': [0.0], 'GSM7136340': [0.0], 'GSM7136341': [0.0], 'GSM7136342': [0.0], 'GSM7136343': [0.0], 'GSM7136344': [0.0], 'GSM7136345': [0.0], 'GSM7136346': [0.0], 'GSM7136347': [0.0], 'GSM7136348': [0.0], 'GSM7136349': [0.0], 'GSM7136350': [0.0], 'GSM7136351': [0.0], 'GSM7136352': [0.0], 'GSM7136353': [0.0], 'GSM7136354': [0.0], 'GSM7136355': [0.0], 'GSM7136356': [0.0], 'GSM7136357': [0.0], 'GSM7136358': [0.0], 'GSM7136359': [0.0], 'GSM7136360': [0.0], 'GSM7136361': [0.0], 'GSM7136362': [0.0], 'GSM7136363': [0.0], 'GSM7136364': [0.0], 'GSM7136365': [0.0], 'GSM7136366': [0.0], 'GSM713

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_Cancer/GSE228783/GSE228783_series_matrix.txt.gz


Gene data shape: (49386, 152)
First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# The identifiers like '11715100_at', '11715101_s_at' are probe IDs from microarray platforms,
# not standard human gene symbols. These need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GeneChip Array', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Transcript ID(Array Design)', 'Target Description', 'Representative Public ID', 'Archival UniGene Cluster', 'UniGene ID', 'Genome Version', 'Alignments', 'Gene Title', 'Gene Symbol', 'Chromosomal Location', 'GB_LIST', 'SPOT_ID', 'Unigene Cluster Type', 'Ensembl', 'Entrez Gene', 'SwissProt', 'EC', 'OMIM', 'RefSeq Protein ID', 'RefSeq Transcript ID', 'FlyBase', 'AGI', 'WormBase', 'MGI Name', 'RGD Name', 'SGD accession number', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function', 'Pathway', 'InterPro', 'Trans Membrane', 'QTL', 'Annotation Description', 'Annotation Transcript Cluster', 'Transcript Assignments', 'Annotation Notes']
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene IDs and gene symbols in the annotation dataframe
# From the gene annotation preview, we can see that:
# - 'ID' column contains identifiers that match the gene expression data (e.g. 11715100_at)
# - 'Gene Symbol' column contains the human gene symbols (e.g. HIST1H3G)

# 2. Create a gene mapping dataframe with these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print("Gene mapping dataframe created with shape:", gene_mapping.shape)
print("Sample of gene mapping data:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data created with shape:", gene_data.shape)
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])


Gene mapping dataframe created with shape: (49384, 2)
Sample of gene mapping data:
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2


Gene expression data created with shape: (19521, 152)
First 5 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(linked_data.iloc[:5, :5])

# Check and rename the trait column if needed
if trait not in linked_data.columns and '0.0' in linked_data.columns:
    # Rename the column to the expected trait name
    linked_data = linked_data.rename(columns={'0.0': trait})
    print(f"Renamed column '0.0' to '{trait}'")

# Verify the trait column exists
if trait not in linked_data.columns:
    print(f"Error: Trait column '{trait}' not found in linked data. Available columns: {linked_data.columns[:10].tolist()}")
    is_usable = False
else:
    # 3. Handle missing values
    print("\nHandling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

    # 4. Check for bias in the dataset
    print("\nChecking for bias in dataset features...")
    trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

    # 5. Conduct final quality validation and save metadata
    print("\nConducting final quality validation...")
    note = "This dataset contains liver tissue gene expression data with samples categorized as HCC (hepatocellular carcinoma) or other conditions (CRC Met, CCC, other)."
    is_gene_available = len(normalized_gene_data) > 0
    is_trait_available = trait in linked_data_clean.columns
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=trait_biased,
        df=linked_data_clean,
        note=note
    )

    # 6. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")


Normalizing gene symbols...
Gene data shape after normalization: (19298, 152)
First 10 gene identifiers after normalization:
['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE228783.csv

Linking clinical and genetic data...
Linked data shape: (152, 19299)
Linked data preview:
            Liver_Cancer      A1BG       A1CF       A2M     A2ML1
GSM7136321           0.0  12.33124  20.535768  12.32219  4.097415
GSM7136322           0.0  12.40018  19.749593  12.34718  4.237847
GSM7136323           0.0  12.22078  19.805032  12.32856  4.280063
GSM7136324           0.0  12.31479  19.829038  12.21530  4.366853
GSM7136325           0.0  12.32009  19.083230  12.43024  4.634051

Handling missing values...


Linked data shape after handling missing values: (152, 19299)

Checking for bias in dataset features...
For the feature 'Liver_Cancer', the least common label is '1.0' with 12 occurrences. This represents 7.89% of the dataset.
The distribution of the feature 'Liver_Cancer' in this dataset is fine.


Conducting final quality validation...


Linked data saved to ../../output/preprocess/Liver_Cancer/GSE228783.csv
